# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end

aluminium_setup (generic function with 2 methods)

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trival statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298429610787                   -0.85    4.0
  2   -8.300212429497       -2.75       -1.25    1.5
  3   -8.300447055036       -3.63       -1.87    3.5
  4   -8.300462255217       -4.82       -2.70    1.1
  5   -8.300464473590       -5.65       -3.23    3.9
  6   -8.300464577611       -6.98       -3.35    9.6
  7   -8.300464605445       -7.56       -3.49    1.0
  8   -8.300464624937       -7.71       -3.63    1.0
  9   -8.300464639204       -7.85       -3.83    1.5
 10   -8.300464642004       -8.55       -3.96    2.5
 11   -8.300464643810       -8.74       -4.28    1.1


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.67477132233                   -0.70    6.8
  2   -16.67874052187       -2.40       -1.14    1.0
  3   -16.67923395519       -3.31       -1.86    4.5
  4   -16.67928018177       -4.34       -2.76    2.5
  5   -16.67928610474       -5.23       -3.21    4.5
  6   -16.67928621596       -6.95       -3.48    4.4
  7   -16.67928622255       -8.18       -3.94    1.0
  8   -16.67928622426       -8.77       -4.62    1.0


In [5]:
self_consistent_field(aluminium_setup(4); is_converged, n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32408038840                   -0.56    6.2
  2   -33.33253069971       -2.07       -1.00    1.0
  3   -33.33576707089       -2.49       -1.69    3.5
  4   -33.33686003188       -2.96       -2.35    4.6
  5   -33.33693270220       -4.14       -2.80   10.5
  6   -33.33694382417       -4.95       -3.74    3.8
  7   -33.33694390138       -7.11       -4.08    7.8


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298278757069                   -0.85    4.5
  2   -8.300252094182       -2.70       -1.59    1.0
  3   -8.300448508350       -3.71       -2.53    7.2
  4   -8.300367274310   +   -4.09       -2.26   12.9
  5   -8.300464371791       -4.01       -3.56    3.5
  6   -8.300464571708       -6.70       -3.88    3.8
  7   -8.300464643226       -7.15       -4.82    2.6


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing(), n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32544297343                   -0.56    6.2
  2   -33.26847730435   +   -1.24       -1.24    1.0
  3   +18.65990520763   +    1.72       -0.21   11.5
  4   -33.27534948681        1.72       -1.46    8.5
  5   -33.21967429967   +   -1.25       -1.35    3.9
  6   -29.45779794265   +    0.58       -0.77    4.8
  7   -33.31345774511        0.59       -1.83    7.4
  8   -33.33578208109       -1.65       -2.23    2.9
  9   -33.33492008440   +   -3.06       -2.10    2.8
 10   -33.33662292313       -2.77       -2.69    2.5
 11   -33.33679772538       -3.76       -2.88    2.0
 12   -33.33689715671       -4.00       -3.13    2.9
 13   -33.33694140988       -4.35       -3.53    3.2
 14   -33.33694063308   +   -6.11       -3.76    3.4
 15   -33.33694363574       -5.52       -4.18    2.6


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.